<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

train_prompts = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_prompts.csv")
print(train_prompts.head())
print("\n")
train_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_essays.csv")
print(train_essays.head())
print("\n")
test_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/test_essays.csv")
print(test_essays.head())
# ai_human_dataset = pd.read_csv("/content/AI_Human.csv")
# print(ai_human_dataset.head())

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2    

In [2]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> all
Command 'all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def _remove_stopwords_(texts):
  return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def preprocess_text(series):
    def process(text):
        tokens = word_tokenize(text)
        tokens = _remove_stopwords_(tokens)
        tokens = lemmatize_text(tokens)
        return " ".join(tokens)
    return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [7]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  
0          0  car Cars around since become famous 1900s Henr...  
1          0  transportation large necessity country worldwi...  
2          0  America love affair vehicle seem cool say Elis...  
3          0  how often ride car do drive one motor vehicle ...  
4          0  car wonderful thing they perhaps one world gre...  


In [8]:
from gensim.models import Word2Vec

sentences = train_essays['preprocessedText'].apply(lambda x: x.split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

# train_essays['word2vecVector'] = sentences.apply(
#     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# )

def calculate_average_word_vector(words,model,vector_size=100):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

train_essays['word2vecVector'] = train_essays['preprocessedText'].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


Vector for 'car': [ 1.2111949  -0.30556297 -0.7253577   1.6476825  -0.05973133 -2.5271766
  2.3698192   0.08536696 -1.6918591   1.0612195   0.36911753  0.937336
  0.02407086  1.6470591  -0.9913625  -0.76151234  0.10906904 -0.95410055
 -1.5863189  -3.0185814   2.0043833   0.4172694  -0.10092472 -0.33763146
 -0.3924448  -1.1022891  -1.208857   -0.21761449 -1.909737   -0.72541547
 -1.1375293   0.80547315  0.58267814  0.17105478  0.4518194  -0.2760079
  0.12584671 -1.1856534   1.0627922  -2.2725573  -0.41128555  0.24149497
 -1.6320343   0.1893235  -0.10478523 -0.3555176  -0.5094247   1.0395855
  2.5929909   1.5383127  -0.1976783   1.1053098   0.4914338   0.5094458
  1.3409163  -0.9554275   0.45697558 -0.18407106 -2.277477    0.38295817
  0.50247586 -0.0343245  -0.05879904 -1.6111666  -0.8870315   1.5619276
 -0.23527811  0.5536205  -0.6408502  -0.21213892  1.0205584   0.10475906
  0.3971731  -2.3980396   1.4803641  -0.5004348   2.3631022   0.2105331
 -1.5404352  -0.70773375  0.1468584   0.4

In [9]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(train_essays['preprocessedText'])]

doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

train_essays['doc2vecVector'] = train_essays.index.map(lambda x: doc2vec_model.dv[str(x)])


In [10]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  \
0          0  car Cars around since become famous 1900s Henr...   
1          0  transportation large necessity country worldwi...   
2          0  America love affair vehicle seem cool say Elis...   
3          0  how often ride car do drive one motor vehicle ...   
4          0  car wonderful thing they perhaps one world gre...   

                                      word2vecVector  \
0  [-0.024536112, 0.050590437, -0.5430817, -0.255...   
1  [0.07706183, 0.0859

In [11]:
from sklearn.model_selection import train_test_split

y = train_essays['generated']

train_index, test_index = train_test_split(train_essays.index, test_size=0.2, random_state=42)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(train_essays['preprocessedText'])

X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
y_train, y_test = y[train_index], y[test_index]


In [13]:
train_essays['word2vecVector'] = train_essays['word2vecVector'].apply(lambda x: np.array(x))

X_word2vec = np.stack(train_essays['word2vecVector'].values)

X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

print(X_train_word2vec.shape)
print(X_test_word2vec.shape)
print(X_train_word2vec)

(1102, 100)
(276, 100)
[[ 0.0480455   0.00245276 -0.51658636 ... -0.5090301   0.31441182
  -0.10226092]
 [-0.07436413  0.2377062  -0.37200594 ... -0.25823748  0.19648382
   0.1243243 ]
 [-0.68955594  0.561735    0.5310993  ... -0.5083269  -0.01480051
   0.17156057]
 ...
 [-0.06672735  0.03907606 -0.3371643  ... -0.6025266   0.28306353
  -0.20842443]
 [-0.86480206  0.5640187   0.64005625 ... -0.43267986 -0.11761905
   0.25580472]
 [-0.0423575   0.36134255 -0.38192007 ... -0.20374025  0.13378517
  -0.06544389]]


In [14]:
train_essays['doc2vecVector'] = train_essays['doc2vecVector'].apply(lambda x: np.array(x))
X_doc2vec = np.stack(train_essays['doc2vecVector'].values)
X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
print(X_train_doc2vec.shape)
print(X_test_doc2vec.shape)
print(X_train_doc2vec)

(1102, 100)
(276, 100)
[[ 0.07147887 -0.32900298 -0.23523858 ... -0.14498188  0.45644414
   0.13322556]
 [-0.28677464 -0.09217716 -0.73470193 ...  0.05696475  0.27619284
   0.3492302 ]
 [-0.46581587  0.05295126  0.2343993  ... -0.41684785  0.09282099
   0.14894862]
 ...
 [-0.04361692 -0.14471887 -0.4125112  ... -0.51393485  0.67461795
   0.04859068]
 [-0.4447485   0.18331645  0.11205757 ... -0.43579936  0.04945527
   0.16391483]
 [-0.02295278  0.02559162 -0.3291557  ...  0.00198629 -0.00682148
  -0.03970782]]


In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_svm_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_svm_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for SVM:", results_tfidf)


results_word2vec = evaluate_svm_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for SVM:", results_word2vec)


results_doc2vec = evaluate_svm_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for SVM:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
from sklearn.tree import DecisionTreeClassifier
def evaluate_decisionTree_model(X_train, X_test, y_train, y_test):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred = dt_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }

results_tfidf = evaluate_decisionTree_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Decision Tree:", results_tfidf)


results_word2vec = evaluate_decisionTree_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Decision Tree:", results_word2vec)


results_doc2vec = evaluate_decisionTree_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Decision Tree:", results_doc2vec)


TF-IDF Results for Decision Tree: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results for Decision Tree: {'Accuracy': 0.9927536231884058, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results for Decision Tree: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
from sklearn.ensemble import RandomForestClassifier
def evaluate_randomForest_model(X_train, X_test, y_train, y_test):
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
results_tfidf = evaluate_randomForest_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Random Forest:", results_tfidf)


results_word2vec = evaluate_randomForest_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Random Forest:", results_word2vec)


results_doc2vec = evaluate_randomForest_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Random Forest:", results_doc2vec)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TF-IDF Results for Random Forest: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Word2Vec Results for Random Forest: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results for Random Forest: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
from sklearn.neighbors import KNeighborsClassifier
def evaluate_knn_model(X_train, X_test, y_train, y_test):
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
results_tfidf = evaluate_knn_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for KNN:", results_tfidf)


results_word2vec = evaluate_knn_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for KNN:", results_word2vec)


results_doc2vec = evaluate_knn_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for KNN:", results_doc2vec)


TF-IDF Results for KNN: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results for KNN: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results for KNN: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
